#### Prepare Cellpose training dataset  

30 training images from the HCC1143 series of experiments. Early and late Untreated plus late high dose from each drug. For nuclear segmentation, apply the red channel as a mask to the phase, then combine them to get the red channel "textured" by the phase images. Use red and phase as separate channels for cytoplasmic segmentation.  Use registered intensity images and prefiltered masks. Masks must have the same name as the intensity images with a suffix added before the file type. 

In [116]:
import numpy as np
import numpy.ma as ma
import os, glob, tifffile
from skimage import io, filters, exposure

In [121]:
def transfer_file(data_path, plateID, pipeline_name, well, field_num, time_slice):
    filename = os.path.join(data_path, plateID,"Analysis",pipeline_name,"intermediate_files","tracking",well,"field_"+str(field_num),"reg","t"+time_slice+".tif")
    print("preparing "+filename+" for training set")
    #read in the registered nuclear channel image
    image = io.imread(filename).astype("uint16")
    #write out registered nuclear channel image with full metadata filename
    out_filename = os.path.join(data_path, "cellpose_training", "image_cache",plateID+"_"+well+"_"+str(field_num)+"_"+time_slice+"_img.tif")
    tifffile.imwrite(out_filename, image)
    #get corresponding mask file and write to disk
    mask_filename = os.path.join(data_path, plateID,"Analysis",pipeline_name,"intermediate_files","tracking",well,"field_"+str(field_num),"filtered_masks","mask"+time_slice+".tif")
    mask = io.imread(mask_filename).astype("uint16")
    mask_out_filename = os.path.join(data_path, "cellpose_training", "image_cache",plateID+"_"+well+"_"+str(field_num)+"_"+time_slice+"_img_masks.tif")
    tifffile.imwrite(mask_out_filename, mask)
    #get and write out slice from phase image stack
    filename = os.path.join(data_path, plateID,"Analysis",pipeline_name,"intermediate_files",plateID+'_P_'+well+'_'+str(field_num)+'_reg_stack.tif')
    p_stack = io.imread(filename)
    p_image = p_stack[int(time_slice)]
    #create a mask of the nuclear image without the baskground
    val = filters.threshold_otsu(image)
    nuc_mask = image<val
    phase_image_masked = np.ma.MaskedArray(p_image, nuc_mask).filled(0)
    image_masked = np.ma.MaskedArray(image, nuc_mask).filled(0)
    np_out_filename = out_filename.replace('_img', '_np_img')
    tifffile.imwrite(np_out_filename, image_masked+phase_image_masked)
    tifffile.imwrite(np_out_filename.replace('_np_','_p_'), p_image)
    

hist, bins_center = exposure.histogram(image)

plt.figure(figsize=(18, 8))
plt.subplot(131)
plt.imshow(image, cmap='gray', interpolation='nearest')
plt.axis('off')
plt.subplot(132)
plt.imshow(nuc_mask, cmap='gray', interpolation='nearest')
plt.axis('off')
plt.subplot(133)
plt.imshow(phase_image_masked, cmap = 'gray', interpolation = 'nearest')
plt.tight_layout()

In [122]:
data_path = "/home/exacloud/gscratch/HeiserLab/images/"
plateID_list = ("HC00701", "HC00801", "HC00901", "HC01001", "HC01301", "HC01401")
#plateID_list = (["HC00701"])
pipeline_name = "CKn"
well_list = ("A1", "A1","D2", "D4", "D6")
time_slice_list = ("004", "188", "188", "188", "188")
#well_list = (["D2"])
field_num = 1


In [123]:
res = list(map(transfer_file, [data_path]*len(well_list)*len(plateID_list),
               [element for element in plateID_list for i in range(len(well_list))],[pipeline_name]*len(well_list)*len(plateID_list),
               well_list*len(plateID_list),
               [field_num]*len(well_list)*len(plateID_list),
               time_slice_list*len(plateID_list)))


preparing /home/exacloud/gscratch/HeiserLab/images/HC00701/Analysis/CKn/intermediate_files/tracking/A1/field_1/reg/t004.tif for training set
preparing /home/exacloud/gscratch/HeiserLab/images/HC00701/Analysis/CKn/intermediate_files/tracking/A1/field_1/reg/t188.tif for training set
preparing /home/exacloud/gscratch/HeiserLab/images/HC00701/Analysis/CKn/intermediate_files/tracking/D2/field_1/reg/t188.tif for training set
preparing /home/exacloud/gscratch/HeiserLab/images/HC00701/Analysis/CKn/intermediate_files/tracking/D4/field_1/reg/t188.tif for training set
preparing /home/exacloud/gscratch/HeiserLab/images/HC00701/Analysis/CKn/intermediate_files/tracking/D6/field_1/reg/t188.tif for training set
preparing /home/exacloud/gscratch/HeiserLab/images/HC00801/Analysis/CKn/intermediate_files/tracking/A1/field_1/reg/t004.tif for training set
preparing /home/exacloud/gscratch/HeiserLab/images/HC00801/Analysis/CKn/intermediate_files/tracking/A1/field_1/reg/t188.tif for training set
preparing /ho